In [169]:
import numpy as np 
import time 
from useful_function_joint import *
import pymanopt
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem
from typing import Sequence

import networkx as nx

In [2]:
e1 = np.array((1,0,0))*10
e2 = np.array((0,1,0))*10
e3 = np.array((0,0,1))*10
e0 = np.array([0,0,0])

In [3]:
t1 = np.array((25,0,0))
t2 = np.array((560,0,0))
t3 = np.array((515,0,0))
t4 = np.array((0,0,0))
t5 = np.array((90,0,0))

In [4]:
joints_ref = np.array([0,-90,90,0,0,0])*np.pi/180

In [5]:
R1 = Rz(joints_ref[0])
R2 = Ry(joints_ref[1])
R3 = Ry(joints_ref[2])
R4 = Rx(joints_ref[3])
R5 = Ry(joints_ref[4])
R6 = Rx(joints_ref[5])

In [6]:
T1 = T(t1)
T2 = T(t2)
T3 = T(t3)
T4 = T(t4)
T5 = T(t5)

In [7]:
axes = ['z','y','y','x','y','x']
translation = [T(e0),T1,T2,T3,T4,T5]
rotation = [R1,R2,R3,R4,R5,R6]

step_vect = {'z': [e0,e3],
             'y': [2*e1,-2*e1],
             'x': [e0,-e1]}

In [8]:
np.sqrt(10**2+1)

10.04987562112089

In [88]:
G = nx.Graph()

G.add_node(1,pos_ref=e1)
G.add_node(2,pos_ref=e2)
G.add_edge(1,2, weight  = np.linalg.norm(e1-e2))


count = 2

for i, ax in enumerate(axes):
    v1,v2 = step_vect[ax]
    
    transform = []
    
    for j in range(i+1):
        transform.append(translation[j])
        transform.append(rotation[j])
    transform = np.array(transform)
    transformation = np.linalg.multi_dot(transform)
    
    G.add_node(count+2,pos_ref=(transformation@T(v1))[:3,3])
    G.add_node(count+1,pos_ref=(transformation@T(v2))[:3,3])
    
    node_pos_ref = nx.get_node_attributes(G,'pos_ref')
    
    G.add_edge(count+2,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count+2]))
    
    G.add_edge(count,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count]))
    G.add_edge(count-1,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count-1]))
    
    G.add_edge(count,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count]))
    G.add_edge(count-1,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count-1]))
    
    count = count + 2

transform = []
for j in range(len(translation)):
    transform.append(translation[j])
    transform.append(rotation[j])
    
#G.add_node(count+1,pos_ref=(transformation@T(e1/10))[:3,3])
#G.add_node(count+2,pos_ref=(transformation@T(e2/10))[:3,3])
#G.add_node(count+3,pos_ref=(transformation@T(e3/10))[:3,3])

#node_pos_ref = nx.get_node_attributes(G,'pos_ref')

#G.add_edge(count+1,count, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count]))
#G.add_edge(count+2,count, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count]))
#G.add_edge(count+3,count, weight = np.linalg.norm(node_pos_ref[count+3]-node_pos_ref[count]))

#G.add_edge(count+1,count-1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count-1]))
#G.add_edge(count+2,count-1, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count-1]))
#G.add_edge(count+3,count-1, weight = np.linalg.norm(node_pos_ref[count+3]-node_pos_ref[count-1]))

mapping = {1: "x", 2: "y", 3: "a1_origin", 4: "a1_plus", 5: "a2_plus", 6: " a2_minus", 7: "a3_plus", 8:"a3_minus", 
           9: "a4_origin", 10:"a4_minus", 11: "a5_plus", 12:"a5_minus", 13: "a6_origin", 14:"a6_minus"}
           
#,15:'x_target',16:'y_target',17:'z_target'}

G = nx.relabel_nodes(G, mapping)


In [10]:
G.nodes
len(weight.keys())
G.edges

NameError: name 'weight' is not defined

In [95]:
G.edges()

EdgeView([('x', 'y'), ('x', 'a1_origin'), ('x', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a1_plus'), ('a1_plus', 'a1_origin'), ('a1_plus', 'a2_plus'), ('a1_plus', ' a2_minus'), ('a1_origin', 'a2_plus'), ('a1_origin', ' a2_minus'), (' a2_minus', 'a2_plus'), (' a2_minus', 'a3_plus'), (' a2_minus', 'a3_minus'), ('a2_plus', 'a3_plus'), ('a2_plus', 'a3_minus'), ('a3_minus', 'a3_plus'), ('a3_minus', 'a4_origin'), ('a3_minus', 'a4_minus'), ('a3_plus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a4_minus', 'a4_origin'), ('a4_minus', 'a5_plus'), ('a4_minus', 'a5_minus'), ('a4_origin', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a5_minus', 'a5_plus'), ('a5_minus', 'a6_origin'), ('a5_minus', 'a6_minus'), ('a5_plus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a6_minus', 'a6_origin')])

In [154]:
pos_ref = nx.get_node_attributes(G,'pos_ref')
M = np.array([pos_ref[i] for i in pos_ref.keys()])
np.linalg.det(C@C.transpose())
C[0,:]

array([-1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [163]:
nx.get_node_attributes(G,'pos_ref')

{'x': array([10,  0,  0]),
 'y': array([ 0, 10,  0]),
 'a1_plus': array([0., 0., 0.]),
 'a1_origin': array([ 0.,  0., 10.]),
 ' a2_minus': array([25.,  0., 20.]),
 'a2_plus': array([ 25.,   0., -20.]),
 'a3_minus': array([ 45.,   0., 560.]),
 'a3_plus': array([  5.,   0., 560.]),
 'a4_minus': array([540.,   0., 560.]),
 'a4_origin': array([530.,   0., 560.]),
 'a5_minus': array([560.,   0., 560.]),
 'a5_plus': array([520.,   0., 560.]),
 'a6_minus': array([630.,   0., 560.]),
 'a6_origin': array([620.,   0., 560.])}

In [177]:

pos_ref = nx.get_node_attributes(G,'pos_ref')

weight = nx.get_edge_attributes(G, "weight")
D = np.diag([weight[i] for i in weight.keys()])
C = incidence_matrix_(G,oriented=True).toarray()
C = np.array(C)

d = 3
n = D.shape[0]-1-6

direction = np.array([1,0,1,0]).reshape(d,1)
direction = direction/np.linalg.norm(direction)

direction_frame_origin_ = [('x', 'y'), ('x', 'a1_origin'), ('x', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a1_plus'), ('a1_plus', 'a1_origin')]
direction_frame_origin = [manifolds.ConstantFactory((pos_ref[in_]-pos_ref[out_]).reshape(d,1)/np.linalg.norm(pos_ref[in_]-pos_ref[out_])) for out_, in_ in direction_frame_origin_]

robot = manifolds.Oblique(d,n)
direction_end_effector = manifolds.ConstantFactory(direction)

list_manifold = direction_frame_origin
list_manifold.append(robot)
list_manifold.append(direction_end_effector)

manifold_ = manifolds.Product(list_manifold)

Q2 = -D@(C.transpose()@np.linalg.pinv(C@C.transpose()))@C@D;
Q1 = D@D;

@pymanopt.function.numpy(manifold_)
def cost(Y1, Y2,Y3,Y4,Y5,Y6,Y7,Y8):
    Y_ = np.concatenate([Y1, Y2,Y3,Y4,Y5,Y6,Y7,Y8],axis=1)
    return np.trace(Q1 + Q2@Y_.transpose()@Y_)

@pymanopt.function.numpy(manifold_)
def euclidean_gradient(Y1, Y2,Y3,Y4,Y5,Y6,Y7,Y8):
    Y_ = np.concatenate([Y1, Y2,Y3,Y4,Y5,Y6,Y7,Y8],axis=1)
    grad = 2*Q2@Y_.transpose()
    return [grad[0,:].reshape(d,1),grad[1,:].reshape(d,1),grad[2,:].reshape(d,1),grad[3,:].reshape(d,1),
            grad[4,:].reshape(d,1),grad[5,:].reshape(d,1),grad[6:-1,:].transpose(),grad[-1,:].reshape(d,1)]
                        
@pymanopt.function.numpy(manifold_)
def euclidean_hessian(Y1, Y2,Y3,Y4,Y5,Y6,Y7,Y8,U1,U2,U3,U4,U5,U6,U7,U8):
    hess = 2*Q2@np.concatenate([U1,U2,U3,U4,U5,U6,U7,U8],axis=1).transpose()
    return [hess[0,:].reshape(d,1),hess[1,:].reshape(d,1),hess[2,:].reshape(d,1),hess[3,:].reshape(d,1),
            hess[4,:].reshape(d,1),hess[5,:].reshape(d,1),hess[6:-1,:].transpose(),hess[-1,:].reshape(d,1)]




problem = Problem(manifold=manifold_, cost=cost,euclidean_gradient=euclidean_gradient, euclidean_hessian=euclidean_hessian)

optimizer = optimizers.TrustRegions(max_iterations=2000)
start = time.time()
Y_star = optimizer.run(problem,initial_point=manifold_.random_point()).point
end = time.time()
end-start

ValueError: cannot reshape array of size 4 into shape (15,1)

In [151]:
X@R.transpose()

array([[ 1.00000022e+01, -3.83692077e-09, -1.85364688e-09],
       [ 9.73477587e-06, -9.83212922e+00, -1.82464196e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.58501008e-04, -1.82483620e+00,  9.83225109e+00],
       [ 4.10904023e+00,  2.05970644e+01,  2.41633748e+01],
       [ 3.67409536e+00,  2.79640146e+01, -1.51500265e+01],
       [-5.01716049e+00, -6.58518749e+01,  5.57495627e+02],
       [ 2.34537903e+01, -9.34091009e+01,  5.52016003e+02],
       [-3.57303000e+02,  2.74652786e+02,  6.28037716e+02],
       [-3.50189738e+02,  2.67802884e+02,  6.26457474e+02],
       [-3.71512795e+02,  2.88273241e+02,  6.31584779e+02],
       [-3.43085442e+02,  2.60995921e+02,  6.24668343e+02],
       [-4.21236419e+02,  3.35907659e+02,  6.44175434e+02],
       [-4.14140031e+02,  3.29129411e+02,  6.42249762e+02]])

In [149]:
from scipy.spatial.transform import Rotation as R

v = np.array([   2.25366279,    5.86244727,    7.78156537])
u = np.cross(v/np.linalg.norm(v),e1/10)

R = rotation_matrix_from_vectors(v/10,e1/10)

In [136]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [145]:
X = np.linalg.pinv(C@C.transpose())@C@D@np.concatenate([Y_star[0].transpose(),Y_star[1].reshape(1,d)],axis=0)
(X[-1,:]-X[-2,:])/np.linalg.norm(X[-2,:]-X[-1,:])

array([ 7.07106215e-01, -4.30210497e-07,  7.07107348e-01])

In [147]:
X = X - X[2,:]
nx.get_node_attributes(G,'pos_ref')

{'x': array([10,  0,  0]),
 'y': array([ 0, 10,  0]),
 'a1_plus': array([0., 0., 0.]),
 'a1_origin': array([ 0.,  0., 10.]),
 ' a2_minus': array([25.,  0., 20.]),
 'a2_plus': array([ 25.,   0., -20.]),
 'a3_minus': array([ 45.,   0., 560.]),
 'a3_plus': array([  5.,   0., 560.]),
 'a4_minus': array([540.,   0., 560.]),
 'a4_origin': array([530.,   0., 560.]),
 'a5_minus': array([560.,   0., 560.]),
 'a5_plus': array([520.,   0., 560.]),
 'a6_minus': array([630.,   0., 560.]),
 'a6_origin': array([620.,   0., 560.])}

In [59]:
X.shape

(14, 3)

In [52]:
G.edges

EdgeView([('x', 'y'), ('x', 'a1_origin'), ('x', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a1_plus'), ('a1_origin', 'a1_plus'), ('a1_origin', 'a2_plus'), ('a1_origin', ' a2_minus'), ('a1_plus', 'a2_plus'), ('a1_plus', ' a2_minus'), ('a2_plus', ' a2_minus'), ('a2_plus', 'a3_plus'), ('a2_plus', 'a3_minus'), (' a2_minus', 'a3_plus'), (' a2_minus', 'a3_minus'), ('a3_plus', 'a3_minus'), ('a3_plus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a3_minus', 'a4_origin'), ('a3_minus', 'a4_minus'), ('a4_origin', 'a4_minus'), ('a4_origin', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a4_minus', 'a5_plus'), ('a4_minus', 'a5_minus'), ('a5_plus', 'a5_minus'), ('a5_plus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a5_minus', 'a6_origin'), ('a5_minus', 'a6_minus'), ('a6_origin', 'a6_minus')])